In [1]:
from nlg.train import parse
import nlg.ordering as ordering
import nlg.structuring as structuring
import nlg.lexicalization as lexicalization
import nlg.reference as reference

In [2]:
from nlg.generation import Generation
from nlg.realization import Realization
from inpe_deter.content import content
from datetime import datetime

from db.model import DeforestationDeterINPE, CoronaVirus
from datetime import datetime

In [3]:
structuring_path = 'inpe_deter/data/month_grammar/structuring.json'
lexicalization_path = 'inpe_deter/data/month_grammar/lexicalization.json'
reference_path = 'inpe_deter/data/month_grammar/references.json'
lexicon_path = 'nlg/lexicons'
gen = Generation(structuring_path, lexicalization_path, reference_path, lexicon_path)

In [4]:
for date in DeforestationDeterINPE.objects().distinct('date')[:5]:
    month, year = date.month, date.year
    messages, date = content(month, year)
    try:
        entry, template, paragraphs = gen.generate(messages, strategy='random')

        text = []
        for p in paragraphs:
            text.append(' '.join(p))
        text = '\n\n'.join(text)

        print("Portuguese:")
        print(text)
    except:
        print('ERROR')
    print(10 * '*', '\n')

Portuguese:
Um total de 914 km² de desmatamento na Amazônia Legal foi detectado no mês de junho de 2019 pelo INPE. Com 835 km², a principal causa de devastação foi o desmatamento com solo exposto, que deixa a terra sem nenhuma cobertura florestal.

O estado que mais teve desmatamento foi Pará (454 km²). O município que teve mais desmatamento no mês foi Altamira (Pará), com 95 km² de área desmatada.

Com 12 km², a FLORESTA NACIONAL DO JAMANXIM / Pará foi a Unidade de Conservação mais devastada no mês.
********** 

Portuguese:
Um total de 914 km² de desmatamento na Amazônia legal foi detectado no mês de junho de 2019 pelo INPE. Com 835 km², a principal causa de devastação foi o desmatamento com solo exposto, aquele que deixa a terra sem nenhuma vegetação.

Pará foi o estado mais atingido pelo desmatamento, com 454 km². Altamira foi o município mais afetado (95 km²).

A Unidade de Conservação onde teve mais desmatamento foi a FLORESTA NACIONAL DO JAMANXIM / Pará (12 km²).
********** 

Por

# Daily Report

In [5]:
from datetime import datetime, timedelta
import inpe_deter.daily_content as daily_content

structuring_path = 'inpe_deter/data/daily_grammar/structuring.json'
lexicalization_path = 'inpe_deter/data/daily_grammar/lexicalization.json'
reference_path = 'inpe_deter/data/daily_grammar/references.json'
lexicon_path = 'nlg/lexicons'
gen = Generation(structuring_path, lexicalization_path, reference_path, lexicon_path)

In [6]:
date = datetime.now() - timedelta(days=60)

alerts = []
for last_day in sorted(DeforestationDeterINPE.objects(date__gte=date).distinct('date'))[:5]:
    cities, ucs = {}, {}
    cases = DeforestationDeterINPE.objects(date=last_day)
    for case in cases:
        if case.cause in ['DESMATAMENTO_CR', 'DESMATAMENTO_VEG', 'MINERACAO']:
            if case.UC:
                if (last_day, case.state, case.city, case.UC) not in ucs:
                    ucs[(last_day, case.state, case.city, case.UC)] = 0
                ucs[(last_day, case.state, case.city, case.UC)] += case.uc_area
            else:
                if (last_day, case.state, case.city, None) not in cities:
                    cities[(last_day, case.state, case.city, None)] = 0
                cities[(last_day, case.state, case.city, None)] += case.city_area

    cities = sorted(cities.items(), key=lambda x:x[1], reverse=True)[:3]
    ucs = sorted(ucs.items(), key=lambda x:x[1], reverse=True)[:3]

    for city in cities:
        alerts.append(city[0])
    for uc in ucs:
        alerts.append(uc[0])

In [7]:
for alert in alerts[:5]:
    date, state, city, uc = alert
    messages, _ = daily_content.content(date, state, city, uc)
    print('\n'.join([msg['str_msg'] for msg in messages]))
    print()
    try:
        entry, template, paragraphs = gen.generate(messages, strategy='random')

        text = []
        for p in paragraphs:
            text.append(' '.join(p))
        print('\n'.join(text))
    except:
        print('ERROR')
    print(10 * '*', '\n')

CAUSE(cause="DESMATAMENTO_CR",city="Porto Velho",location="deter-amz",month="6",state="RO",year="2020")
DAILY_ALERT(area="6.7931018036451825",city="Porto Velho",daily_accumulation="9",day="30",location="deter-amz",month="6",state="RO",year="2020")
TOTAL_DEFORESTATION(area="46.0882973437658",city="Porto Velho",location="deter-amz",month="6",state="RO",year="2020")

No dia 30 de junho de 2020, o INPE registrou alertas de desmatamento somando 6,79 km² em Porto Velho / Rondônia, que acumula 9 dias com alertas no mês. O principal tipo de desmatamento foi o desmatamento com solo exposto, que deixa o solo sem vegetação. No total, 46,09 km² foram desmatados em Porto Velho no mês de junho.
********** 

CAUSE(cause="DESMATAMENTO_CR",city="Sao Felix do Xingu",location="deter-amz",month="6",state="PA",year="2020")
DAILY_ALERT(area="6.161107067840259",city="Sao Felix do Xingu",daily_accumulation="11",day="30",location="deter-amz",month="6",state="PA",year="2020")
TOTAL_DEFORESTATION(area="67.975544